# Usage Example

This notebook matches `examples/example.py`: load intervals, run pipeline, write parquet, and create quick plots.


In [ ]:
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import polars as pl

from chameleon_usage.ingest import load_intervals
from chameleon_usage.ingest.coerce import clamp_hierarchy
from chameleon_usage.pipeline import resample, run_pipeline
from chameleon_usage.schemas import PipelineSpec

In [ ]:
DATA_DIR = "data/current/chi_tacc"
TIME_RANGE = (datetime(2024, 1, 1), datetime(2024, 12, 31))
BUCKET = "1d"

OUTPUT_DIR = Path("output/example/")
OUTPUT_FILE = OUTPUT_DIR / "usage_timeline.parquet"
PLOTS_DIR = OUTPUT_DIR / "plots"

SPEC = PipelineSpec(group_cols=("metric", "resource"), time_range=TIME_RANGE)

In [ ]:
intervals = load_intervals(DATA_DIR, TIME_RANGE)
intervals = intervals.collect().lazy()

cleaned_intervals, invalid_intervals = clamp_hierarchy(intervals)
cleaned_intervals = cleaned_intervals.collect().lazy()

usage = run_pipeline(cleaned_intervals, SPEC)
resampled = resample(usage, BUCKET, SPEC)
thunk = resampled.collect()

thunk.select("timestamp", "metric", "resource", "value").head(20)

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
thunk.write_parquet(OUTPUT_FILE)
print(f"Wrote parquet: {OUTPUT_FILE}")

In [ ]:
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

metrics = [
    "total",
    "reservable",
    "committed",
    "occupied_reservation",
    "occupied_ondemand",
]

for resource, title in [("nodes", "Nodes"), ("vcpus", "vCPUs")]:
    frame = (
        thunk.filter(pl.col("resource") == resource)
        .filter(pl.col("metric").is_in(metrics))
        .pivot(on="metric", index="timestamp", values="value")
        .sort("timestamp")
    )
    if frame.is_empty():
        continue

    x = frame.get_column("timestamp").to_list()

    fig, ax = plt.subplots(figsize=(9, 4))
    for metric in metrics:
        if metric not in frame.columns:
            continue
        y = frame.get_column(metric).to_list()
        ax.plot(x, y, label=metric)

    ax.set_title(f"{title} usage ({BUCKET})")
    ax.set_xlabel("timestamp")
    ax.set_ylabel(title)
    ax.grid(alpha=0.3)
    ax.legend()
    fig.autofmt_xdate()
    fig.tight_layout()
    fig.savefig(PLOTS_DIR / f"{resource}_quick.png", dpi=200)
    plt.show()
    plt.close(fig)

print(f"Wrote plots: {PLOTS_DIR}")